<font color="#483D8B">
<h1  align="center">Topic Extraction
</h1>
<div align="center">
<font size=3><b>
<br>Ruobing Wang
<br>March 8, 2019
<br></font></b></div>

## Overview
Objectives:

We will use the dataset from sklearn which is related with The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The dataset includes approximately 20,000 newsgroup documents, across 20 different newsgroups. We want to extract the 10 main topics from the dataset by using NMF and LDA methods. 

### Lab Objectives

- Topic extraction analysis
- Explain techniques and concepts used in code example
- Non-negative Matrix Factorization and Latent Dirichlet Allocation


Reference:
https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html






-------------

## Data

The data is related with The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.

 <br/>


What is inside:
It has 20 different news organizations and each mpas the different themes. Some of them are very closely to each other and others are highly unrelated. 

The followings are the list of the 20 newsgroups:

-----------------------------
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	

------------------------------
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	

-----------------------------
sci.crypt
sci.electronics
sci.med
sci.space

-----------------------------
misc.forsale	

-----------------------------
talk.politics.misc
talk.politics.guns
talk.politics.mideast	

-----------------------------
talk.religion.misc
alt.atheism
soc.religion.christian



- The background information of the data can be found at https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html#newsgroups



In [7]:
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 1.167s.


Briefly summary of the dataset:

In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset="train", shuffle = True)
newsgroups_test = fetch_20newsgroups(subset="test", shuffle = True)
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Looking from the dataset, it has a few broad topics like:

- Science
- Politics
- Sports
- Religion
- Technology etc.


---------------


## Exploratory Data Analysis

In [6]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



At first, Load the 20 newsgroups dataset and vectorize it. We use a few heuristics to filter out useless terms early on: the posts are stripped of headers, footers and quoted replies, and common English words, words occurring in only one document or in at least 95% of the documents are removed.

Creating a bag of words matrix, we use Scikit learn. A tf- idf can be applied to the bag of words matrix that NMF must process with the TfidfVectorizer. LDA, using a probabilistic graphical model which only requires raw counts, so a CountVectorizer is used. n features meands the bag of words matrix is restricted to the top 1000.

In [8]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()


Extracting tf-idf features for NMF...
done in 0.317s.
Extracting tf features for LDA...
done in 0.246s.



-------------

## Models

### For the tf-idf features (NMF method):

tf-idf(term frequency–inverse document frequency) intends to reflect how important is to documents. And for the TfidfVectorizer function, at first, we want the times of words appear not too much or too rare. It is because that for the words appear too much, the word maybe, such like 'I' or 'he', which is not meaningful for us. So he set the min_df = 2 and max_df = 0.95 which means we set 95% as our boundary and I believe the max_df has the similar function with srop_words.

Also, for the features:  only consider the top 1000 features ordered by term frequency across the documents.

And for inverse document frequency factor, which will diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.

In [9]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


Extracting tf-idf features for NMF...
done in 0.268s.


### NMF:

NMF is a linear algeabreic model, factors high dimensional vectors into a low dimensionality representation. It takes advantage of the fact that the vectors are non negative. It forces the coefficients be npn negative.

The idea of NMF: V=WH (W weight matrix, H feature matrix, V original matrix), by computing two different matrices from the original matrix to extract weights and features v will contain the weight of words and the columns will be the documents. An algorithm that belongs to an unsupervised learning, where the constraint is that all elements in W and H are greater than zero.

Finding the differences between the topics:

#### Frobenius norm:

$${\displaystyle \|A\|_{\rm {F}}={\sqrt {\sum _{i=1}^{m}\sum _{j=1}^{n}|a_{ij}|^{2}}}={\sqrt {\operatorname {trace} \left(A^{*}A\right)}}={\sqrt {\sum _{i=1}^{\min\{m,n\}}\sigma _{i}^{2}(A)}},} $$

$$||A||_F^2 = \sum_{i,j} A_{ij}^2$$

For the NMF function, n_compinents are the topics, here we have 10 topics and we set the random seed (random_state) equals to 1.

For the default of the loss function which is Frobenius norm. We are finding the shortest distance between the feartures in a topics.

In [10]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)



Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.234s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope program mac email home contact blood
Topic #6: file problem files format win sound ftp pub read save sit

### Kullback- Leibler 

Here, we set the loss function is 'kullback-leibler', and we have the multiplicate- update.

By using this method we are accumulating the distance between the sparse martrics(tfidf) and the matrix X which is the two non-negative matrices (W, H) whose product approximates the non- negative matrix.

KL divergence(relative entropy) is also a non-negative asymmetric to evaluate the difference between 2 probalitity dustributions.

It is commonly used to measure loss in machine learning and measure of how one probability distribution is different from a second reference probability distribution. 

$${\displaystyle D_{\text{KL}}(P\parallel Q)=\sum _{x\in {\mathcal {X}}}P(x)\log \left({\frac {P(x)}{Q(x)}}\right).}$$

In [11]:
# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)



Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.029s.

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: people just like time don say really know way things make think right said did want ve probably work years
Topic #1: windows thanks using help need hi work know use looking mail software does used pc video available running info advance
Topic #2: god does true read know say believe subject says religion mean question point jesus people book christian mind understand matter
Topic #3: thanks know like interested mail just want new send edu list does bike thing email reply post wondering hear heard
Topic #4: time new 10 year sale old offer 20 16 15 great 30 weeks good test model condition 11 14 power
Topic #5: use number com government new university data states information talk phone right including security provide control following long used research
Topic #6: edu try file so

### For the tf features (LDA method):

tf(term frequency) also intends to reflect how important is to documents. And for the CountVectorizer function, the parameters are basically have the same meaning.
However, 
since it does not have the idf, which means will not weight down the frequency. 

In [12]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 0.269s.



### LDA:

LDA(latent Dirichlet allocation) with term frequency. Basically, it can let obeservations to be explained by unobservered groups. And explain why some of the parts of the data are similar. The model can be estimated by 2 matrics, which discribes the probability of a selected part.

LDA is an unsupervised machine learning model which can take a document as input and find topics as output. It is used to classify text in a document to a particular topic. It uses two probability values words topics and topics documents. 

Because LDA requires data in the form of integer counts, modifying feature values using TF-IDF and combine them will not fit well.

In [13]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 2.950s.

Topics in LDA model:
Topic #0: edu com mail send graphics ftp pub available contact university list faq ca information cs 1993 program sun uk mit
Topic #1: don like just know think ve way use right good going make sure ll point got need really time doesn
Topic #2: christian think atheism faith pittsburgh new bible radio games alt lot just religion like book read play time subject believe
Topic #3: drive disk windows thanks use card drives hard version pc software file using scsi help does new dos controller 16
Topic #4: hiv health aids disease april medical care research 1993 light information study national service test led 10 page new drug
Topic #5: god people does just good don jesus say israel way life know true fact time law want believe make think
Topic #6: 55 10 11 18 15 team game 19 period play 23 12 13 flyers 20 25 22 17 24 16
Topic #7: car year just cars new engine like bike good oil i

We have 10 componets means we have 10 topics and since the max iterations equals to 5 which means we repeat 5 times. 

For the learning_method, we have 2 method: Batch or Online.

Batch: the only thing we care about is the current iteration.
Online: Rather than rewrite, we mutating stuffs, thus the previous steps matter.

For the learning_offset, basically, it will let the iteration starts gently and graduately.

## Model Evaluation

In [18]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:"% (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
people just like time don say really know way things
Topic 1:
windows thanks using help need hi work know use looking
Topic 2:
god does true read know say believe subject says religion
Topic 3:
thanks know like interested mail just want new send edu
Topic 4:
time new 10 year sale old offer 20 16 15
Topic 5:
use number com government new university data states information talk
Topic 6:
edu try file soon remember problem com program hope mike
Topic 7:
year world team game play won win games season maybe
Topic 8:
think don drive need hard make people mac read going
Topic 9:
just good use way got like ll doesn want sure
Topic 0:
edu com mail send graphics ftp pub available contact university
Topic 1:
don like just know think ve way use right good
Topic 2:
christian think atheism faith pittsburgh new bible radio games alt
Topic 3:
drive disk windows thanks use card drives hard version pc
Topic 4:
hiv health aids disease april medical care research 1993 light
Topic 5:
god people doe

The derived topics from NMF and LDA are displayed above, LDA for the 20 newsgroup dataset has noise data for example topic 6, and some topics are hard to interpret. From the result, I would like to say, the NMF have more meaningful topics.

## Conclusion:

From the result, I think we cannot get the certain answer. Since here, we only use 10 topics (20 in total for the original), we will not any specific metric to evaluate how well the model is performed because they have the similar groups.

The structure of the resulting matrices returned by both NMF and LDA is the same which is great and allows for the python method so we can display the top words in a topic.

Also, we may can from some keywords such like Christian in the topic 2 which we show above to refer this topic should from religion and Christian topic. For the future, I will try to utilize pyLDAvis to visualize model.